# Analysis of the HilbertCube1 sweep results

This analysis will:
1- Load the csv results files on the shape comparison of this analysis
2- Compare the error results with the differents metrics

In [7]:
import numpy
import plotly
from pathlib import Path
from tqdm import tqdm

## Variables

In [5]:
DATA_PATH = Path(r"C:\Users\Cardiff\OneDrive - Cardiff University\Results\HilbertCube1")

## Data loading

In [6]:
csv_files = DATA_PATH.glob("*.csv")
csv_files = list(csv_files)
csv_files

[WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_08_15_23_simple_gnn_1_0_0lr_1e-03 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_08_16_49_simple_gnn_1_0_0lr_1e-04 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_20_18_49_simple_gnn_0.7_0.15_0.15lr_1e-03 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_21_09_14_simple_gnn_1_0_0lr_1e-03 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_21_23_39_double_head_simple_gnn_1_0_0lr_1e-03 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_22_14_23_simple_conv_1_0_0lr_1e-03 test_output.csv'),
 WindowsPath('C:/Users/Cardiff/OneDrive - Cardiff University/Results/HilbertCube1/2023_03_23_05_24_double_head_simple_con